Working on adding a lot of features just to see if it can get the score up regardless of how complicated or where the data is coming from

Features that I will be adding

* Taget given surface counts
* words from the target and source document
  * possible back prop into these vectors, idea is to replace tf-idf with some nn and back prop here
* using a linear layer near the output to combine the features

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer, WordTokenizer
import json
import re

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki2.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links7-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirect7.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [7]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [8]:
len(wordvectors.vectors)

4056055

In [9]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-surface-counts7.json') as f:
    surface_counts = json.load(f)

In [10]:
# try and make the surfaces items match what we are looking for
surface_counts_re = re.compile('([\.,!\?])')
for sk in surface_counts.keys():
    nsk = sk.replace('(', '-lrb-').replace(')', '-rrb-')
    nsk = surface_counts_re.sub(' \\1', nsk)
    if nsk != sk:
        surface_counts[nsk] = surface_counts[sk]

In [11]:
from wikireader import WikiRegexes, WikipediaReader

In [12]:
def PreProcessedQueries(wikipedia_dump_fname, vectors=wordvectors, queries=queries, redirects=page_redirects, surface=surface_counts):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    wordvec = WordTokenizer(vectors, sentence_length=200)
    documentvec = WordTokenizer(vectors, sentence_length=1)
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                documentvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            documentvec.get_location(redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

#     class GetWikipediaWords(WikipediaReader, WikiRegexes):

#         def readPage(ss, title, content):
#             tt = ss.convertToTitle(title)
#             if tt in queried_pages:
#                 cnt = ss._wikiToText(content)
#                 page_content[tt] = wordvec.tokenize(cnt)

#     GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content
    rs = surface

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        documentvecs = documentvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        surface_counts = rs
        
        
    return PreProcessedQueriesCls

In [13]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [14]:
len(wordvectors.vectors)

4056055

In [49]:
class EntityVectorLinkExp(basePreProcessedQueries):

    batch_size = 1000 #20000
    num_training_items = 500000 #200000

    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.document_length = 100
        self.num_words_to_use_conv = 5

        self._setup()


    def _setup(self):
        #self.x_document_input = T.imatrix('x_doc')

        #self.x_document_id = T.ivector('x_doc_id')
        self.x_surface_text_input = T.imatrix('x_surface_link')
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  # TODO

        self.x_target_input = T.ivector('x_target')
        self.x_target_words = T.imatrix('x_target_words')
        self.x_matches_surface = T.ivector('x_match_surface')
        self.x_matches_counts = T.imatrix('x_matches_counts')
        self.x_link_id = T.ivector('x_link_id')

        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')


        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        self.embedding_W_docs = theano.shared(self.documentvecs.get_numpy_matrix().astype(theano.config.floatX))
        
#         self.document_l = lasagne.layers.InputLayer(
#             (None,self.document_length),
#             input_var=self.x_document_input
#         )

#         self.document_embedding_l = EmbeddingLayer(
#             self.document_l,
#             W=self.embedding_W,
#             add_word_params=False,
#         )

#         self.document_conv1_l = lasagne.layers.Conv2DLayer(
#             self.document_embedding_l,
#             num_filters=500,
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             name='document_conv1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_max_l = lasagne.layers.Pool2DLayer(
#             self.document_conv1_l,
#             name='document_pool1',
#             pool_size=(self.document_length - self.num_words_to_use_conv, 1),
#             mode='sum',
#         )

#         self.document_dens1 = lasagne.layers.DenseLayer(
#             self.document_max_l,
#             num_units=250,
#             name='doucment_dens1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop1 = lasagne.layers.DropoutLayer(
#             self.document_dens1,
#             p=.25,
#         )

#         document_output_length = 200

#         self.document_dens2 = lasagne.layers.DenseLayer(
#             self.document_drop1,
#             num_units=225,
#             name='document_dens2',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop2 = lasagne.layers.DropoutLayer(
#             self.document_dens2,
#             p=.25,
#         )

#         self.document_dens3 = lasagne.layers.DenseLayer(
#             self.document_drop2,
#             num_units=document_output_length,
#             name='document_dens3',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_output = lasagne.layers.get_output(self.document_dens3)

        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )

        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )

        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_text_input
        )

        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )

        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_merged_l = lasagne.layers.ConcatLayer(
            [self.surface_context_pool1_l, self.surface_pool1_l]
        )

        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_merged_l,
            name='surface_dens1',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_drop1 = lasagne.layers.DropoutLayer(
#             self.surface_dens1,
#             p=.25,
#         )

#         self.surface_dens2 = lasagne.layers.DenseLayer(
#             self.surface_drop1,
#             name='surface_dens2',
#             num_units=200,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_aligned_l = InputLayer(
#             (None, document_output_length),
#             input_var=self.document_output[self.x_document_id,:]
#         )

        ##############################################
        ## changed to not use the documented

#         self.source_l = lasagne.layers.ConcatLayer(
#             [self.document_aligned_l, self.surface_dens1]
#         )

        self.source_dens1 = lasagne.layers.DenseLayer(
            self.surface_dens1,   # CHANGED
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )

        self.source_dens12 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=250,
            name='source_dens12',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop12 = lasagne.layers.DropoutLayer(
            self.source_dens12,
            p=.25,
        )

        compared_vector_size = self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text

        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop12,
            num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.linear,
        )

        self.source_out = lasagne.layers.get_output(self.source_dens2)

        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)

        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )

        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_matched_counts_input_l = lasagne.layers.InputLayer(
            (None,5),
            input_var=self.x_matches_counts.astype(theano.config.floatX),
        )

        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W_docs,
            add_word_params=False,
        )

        self.target_combined_feats_l = lasagne.layers.ConcatLayer(
            [self.target_embedding_l, self.target_matched_surface_input_l,
            lasagne.layers.reshape(self.target_matched_counts_input_l, ([0],1,1,[1]))],
            axis=3
        )

        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_words,
        )

        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=350,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.target_merge_l = lasagne.layers.ConcatLayer(
            [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
             lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))]
        )

        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_merge_l,
            name='target_wrds_dens1',
            num_units=400,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )

        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_wrds_dens1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )

        self.target_simple = lasagne.layers.DenseLayer(
            self.target_embedding_l, #self.target_combined_feats_l,
            name='target_simple1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )



        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)


#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input],
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)


        #self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)

        self.target_out = lasagne.layers.get_output(self.target_simple)

        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]

        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()

        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)

        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
        
        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)
        
        #############################
        ## Linear features combined
        #############################
        
        
        self.linear_features_combined = lasagne.layers.concat(
            [lasagne.layers.InputLayer(
                    (None, 1), 
                    input_var=self.res_l.reshape((self.res_l.shape[0], 1)),
                ),
            lasagne.layers.reshape(self.target_matched_surface_input_l, ([0],1)),
            self.target_matched_counts_input_l],
            axis=1
        )
        
        self.linear_features_dens_l = lasagne.layers.DenseLayer(
            self.linear_features_combined,
            nonlinearity=lasagne.nonlinearities.linear,
            num_units=1,
            name='linear_final_l',
            W=lasagne.init.Normal(mean=1.0),
        )
        
        self.linear_output = lasagne.layers.get_output(
            lasagne.layers.reshape(self.linear_features_dens_l, ([0],))
        )
        
        ########################################
        ## true output values
        ########################################
        
        self.true_output = self.linear_output
        
        
        
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) *
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))


        #self.golds = self.res_cap[self.y_answer]

#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])

#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)

        def setSubSelector(indx, outputs):
            return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)

        num_target_samples = self.linear_output.shape[0]

        select_seq = T.concatenate([
            self.y_grouping,
            T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
        ], axis=1)

        self.selecting_matrix, _ = theano.scan(
            setSubSelector,
            outputs_info=T.zeros((num_target_samples, num_target_samples)),
            #n_steps=self.y_grouping.shape[0]
            sequences=select_seq,
        )

        self.groupped_elems = T.dot(self.selecting_matrix[-1], 
                                    T.diag(T.exp(self.true_output)))
        self.groupped_res = T.log(self.groupped_elems.sum(axis=0)[T.arange(self.y_grouping.shape[0])])
        self.loss_vec = self.groupped_res - self.true_output[self.y_grouping[:,2]]

        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff,
            lasagne.layers.get_all_params(self.target_simple) +
            lasagne.layers.get_all_params(self.source_dens2) +
            lasagne.layers.get_all_params(self.linear_features_dens_l)
            #lasagne.layers.get_all_params(self.document_dens2)
        )

        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))

        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)

        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?

        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)

        #self.loss_vec = self.grouped

        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)

        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)

        self.func_inputs = [
            #self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, #self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_matches_counts, self.x_link_id,
            self.y_answer, self.y_grouping
        ]  # self.x_target_words,

        ################################################################3
        ## TODO: need to return the actual output layer instead of the res_cap, since that is something else now
        
        self.train_func = theano.function(
            self.func_inputs,
            [self.true_output, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates,
            on_unused_input='ignore',
        )

        self.test_func = theano.function(
            self.func_inputs,
            [self.true_output, self.loss_vec.sum(), self.loss_vec],
            on_unused_input='ignore',
        )

    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.current_learning_groups = []
        self.learning_targets = []
        self.current_surface_target_counts = []
        
        self.failed_match = []
            
    def compute_batch(self, isTraining=True, useTrainingFunc=True):
        if isTraining and useTrainingFunc:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0

        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')

        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink))
                surmatch = surlink.lower()
                surcounts = self.surface_counts.get(surmatch)
                if not surcounts:
                    self.failed_match.append(surmatch)
                    surcounts = {}
                #target_page_input = []
                target_words_input = []
                target_matches_surface = []
                target_inputs = []
                target_learings = []
                target_match_counts = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
                for target in targets['vals'].keys():
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    #cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    wiki_title = WikiRegexes.convertToTitle(target)
                    cnt = self.documentvecs.get_location(wiki_title)
                    if cnt is None:
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    #target_page_input.append(cnt)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target)))
                    target_inputs.append(cnt)  # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                    target_match_counts.append(surcounts.get(wiki_title, 0))
                    #if wiki_title not in surcounts:
                    #    print surcounts, wiki_title
                if target_gold_loc is not None or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    sorted_match_counts = [-4,-3,-2,-1] + sorted(set(target_match_counts))
                    #print sorted_match_counts
                    target_match_counts_indicators = [
                        [
                            int(s == sorted_match_counts[-1]),
                            int(s == sorted_match_counts[-2]),
                            int(s == sorted_match_counts[-3]),
                            int(s <= sorted_match_counts[-4]),
                            int(s != 0),  # if there is at least one occurance of this link with the title
                        ]
                        for s in target_match_counts
                    ]
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input   # TODO: add
                    self.current_target_matches_surface += target_matches_surface
                    self.current_surface_target_counts += target_match_counts_indicators
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])

                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                return
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links

        if len(self.current_target_id) > 0:
            self.run_batch(func)

        return self.total_loss / self.total_links

    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            #self.current_documents,
            self.current_surface_link, self.current_surface_context, #self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id,
            self.current_target_goal, self.current_learning_groups,
            # self.current_target_words,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            l[0]['vals'][ l[1] ] = float(res_vec[i])
        self.reset_accums()

    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')



queries_exp = EntityVectorLinkExp()

In [17]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if all_trained > numSamples:
                        break
                    all_measured += 1
                    all_trained += len(en['vals'].values())
                    m = max(en['vals'].values())
                    if en['vals'][en['gold']] == m and m != 0:
                        all_correct += 1
           
    r = all_measured, float(all_correct) / all_measured
    print r
    return r

In [18]:
import random
def augmentTrainingData():
    for quk in queries.keys():
        qu = queries[quk]
        for enk in qu.keys():
            en = qu[enk]
            if not en['gold']:
                del qu[enk]
        if not qu:
            del queries[quk]
    for qu in queries.values():
        training = random.random() > .15
        for en in qu.values():
            en['training'] = training
augmentTrainingData()

In [19]:
def findWrongItems(trainingData=True, numSamples=50):
    ret = {}
    for qu in queries.values():
        for ek, en in qu.items():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if len(ret) > numSamples:
                        return ret
                    m = max(en['vals'].values())
                    g = en['vals'][en['gold']]
                    if g != m and g != 0:
                        ret[ek] = en
    return ret

In [45]:
queries_exp.check_params()

In [44]:
queries_exp.num_training_items = 50000

In [50]:
%time print queries_exp.compute_batch()

None
CPU times: user 90.2 ms, sys: 2.12 ms, total: 92.3 ms
Wall time: 70.7 ms


In [97]:
gg_func = theano.function(
    queries_exp.func_inputs,
    [queries_exp.true_output, queries_exp.loss_vec.sum(), queries_exp.loss_vec,
     queries_exp.groupped_elems, queries_exp.selecting_matrix[-1],
     queries_exp.y_grouping, queries_exp.true_output,
    queries_exp.groupped_res, queries_exp.true_output[queries_exp.y_grouping[:,2]]],
    #updates=self.updates,
    on_unused_input='ignore',
)





In [98]:
def run_gg(self):
    return gg_func(
        #self.current_documents,
            self.current_surface_link, self.current_surface_context, #self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id,
            self.current_target_goal, self.current_learning_groups,
            # self.current_target_words,
    )

gg_res = run_gg(queries_exp)

gg_res

[array([ 1.98646212,  2.99712896,  2.04826808, ...,  2.04861689,
         2.10181952,  2.10181952], dtype=float32),
 array(110.22624206542969, dtype=float32),
 array([  1.82254648e+00,   2.79867506e+00,   2.80659580e+00,
          7.30113268e-01,   2.46672773e+00,   1.62925196e+00,
          1.14250851e+00,   1.86326838e+00,   3.32961559e-01,
          9.91927624e-01,  -4.45891619e-01,   2.48103142e-02,
          1.43147945e+00,   1.84977865e+00,   8.16366673e-01,
          1.84083939e+00,   2.72906828e+00,   1.79096031e+00,
         -6.90696239e-01,   7.42265940e-01,   9.07172918e-01,
         -9.38940525e-01,   9.91068602e-01,   2.91202927e+00,
          1.02575922e+00,   7.66606808e-01,   7.67705441e-01,
          7.19900370e-01,   1.70493531e+00,   1.90156198e+00,
          8.60829353e-01,   1.91972661e+00,   1.88724828e+00,
          2.97550917e-01,   1.87408769e+00,   3.78248453e+00,
          2.06545448e+00,   6.91780329e-01,   9.24817801e-01,
          3.19688797e+00,   2.74615

In [83]:
np.where(gg_res[-2:][0] - gg_res[-2:][1] < 0)

(array([10, 18, 21, 62, 76]),)

In [84]:
gg_res[-4][10]

array([140, 144, 141], dtype=int32)

In [85]:
gg_res[-2:][1][10]

2.9241138

In [131]:
np.where(np.exp(gg_res[-3]) == 2.97148943)

(array([10]),)

In [115]:
gg_res[-5][:,10]

1.0

In [123]:
gg_res[-6][:,10][140:144]

array([ 2.97148943,  2.97148943,  2.97148943,  2.97148943], dtype=float32)

In [ ]:
len(queries_exp.current_surface_target_counts)

In [ ]:
evalCurrentState(False, 500000)

In [ ]:
evalCurrentState(True, 500000)

In [41]:
exp_results = []

for i in xrange(3):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

(0, -0.08257215597738024)
(1, -0.11069699509292312)
(2, -0.12883317005099618)
(271, 0.7564575645756457)
(268, 0.7761194029850746)


In [47]:
exp_results

[(0, 0.078538599043427942),
 (1, 0.013858774709251018),
 (2, -0.07343112753042555),
 (3, -0.18453017205599387),
 (4, -0.314641321710036),
 (5, -0.46106892619986317),
 (6, -0.62041261969878891),
 (7, -0.78368106992379538),
 ('testing run', -0.53842870805970999),
 ('training state', (2914, 0.5748112560054908)),
 ('testing state', (2765, 0.49330922242314645))]

In [ ]:
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

In [ ]:
for i in xrange(4):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)

In [23]:
queries_exp.all_params[-2:]

[linear_final_l.W, linear_final_l.b]

In [48]:
queries_exp.all_params[-2:][0].get_value(borrow=True)

array([[ 11.81728172],
       [  2.58984065],
       [  2.68953562],
       [  0.09458681],
       [  0.50670612],
       [ -0.62219417],
       [  2.4768362 ]], dtype=float32)

In [ ]:
%time for i in range(5): print queries_exp.compute_batch()

In [ ]:
queries.values()[0]